In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
PROJECT_ID = <your-project-id>
! gcloud config set project $PROJECT_ID
REGION = "us-central1"
BUCKET_NAME = REPOSITORY_NAME = "lh-sandbox"
BUCKET_URI = f"gs://{BUCKET_NAME}"
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI
! gsutil ls -al $BUCKET_URI
OUTPUT_IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY_NAME}/gpt_deployment:latest"
MODEL_DISPLAY_NAME = "gpt"
DIR_SRC = "src"
DIR_MODEL_ARTIFACTS = "model_artifacts" # required files genereated in gpt.ipynb: tiktoken_config.pkl, model_config.pkl, gpt.pth
!gsutil cp {DIR_MODEL_ARTIFACTS}/* {BUCKET_URI}/{DIR_MODEL_ARTIFACTS}/
!gsutil ls {BUCKET_URI}/{DIR_MODEL_ARTIFACTS}/

In [ ]:
%%time
import os

from google.cloud.aiplatform.prediction import LocalModel
from src.gcp_predictor import GPTPredictor  # Update this path as the variable $DIR_SRC to import the custom predictor.

# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.prediction.LocalModel
local_model = LocalModel.build_cpr_model(
    DIR_SRC, # everything here is copied to image
    OUTPUT_IMAGE_URI, # final output image
    predictor=GPTPredictor,
    # handler=, # use default
    requirements_path=os.path.join(DIR_SRC, "deploy_requirements.txt"),
    # no_cache=True, # to rebuild everything
    # base_image=, # base_image ENTRYPOINT/CMD will be overriden
)
# to debug: docker run -it --entrypoint /bin/bash <image>

In [ ]:
local_model.get_serving_container_spec()

In [ ]:
# write a JSON prediciton request
# if timeout error, reduce the size of instances array
!echo '{"instances": ["linsu", "han"]}' > instances.json

In [ ]:
%%time
# local deployment, local storage
with local_model.deploy_to_local_endpoint(artifact_uri=f"{DIR_MODEL_ARTIFACTS}") as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file="instances.json",
        headers={"Content-Type": "application/json"},
    )
    health_check_response = local_endpoint.run_health_check()

In [ ]:
predict_response, predict_response.content

In [ ]:
health_check_response, health_check_response.content

In [ ]:
%%time
# local deployment, gcp storage
with local_model.deploy_to_local_endpoint(artifact_uri=f"{BUCKET_URI}/{DIR_MODEL_ARTIFACTS}") as local_endpoint: #, credential_path=CREDENTIALS_FILE
    predict_response = local_endpoint.predict(
        request_file="instances.json",
        headers={"Content-Type": "application/json"},
    )
    health_check_response = local_endpoint.run_health_check()

In [ ]:
predict_response, predict_response.content

In [ ]:
health_check_response, health_check_response.content

In [ ]:
local_model.push_image()

In [ ]:
%%time
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{DIR_MODEL_ARTIFACTS}",
)

In [ ]:
%%time
endpoint = model.deploy(machine_type="n1-standard-4")

In [ ]:
%%time
# prediction using python
endpoint.predict(instances=["test", "hi"])

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
%%time
# prediction using cli
!gcloud ai endpoints predict $ENDPOINT_ID --region $REGION --json-request instances.json

In [ ]:
%%time
# using REST
!curl -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" -d @instances.json https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:pred

In [ ]:
# # Cleanup

# # Undeploy model and delete endpoint
# endpoint.delete(force=True)

# # Delete the model resource
# model.delete()

# # Delete the container image from Artifact Registry
# !gcloud artifacts docker images delete \
#     --quiet \
#     --delete-tags \
#     {OUTPUT_IMAGE_URI}
# delete_bucket = False

# if delete_bucket or os.getenv("IS_TESTING"):
#     ! gsutil rm -r $BUCKET_URI

https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/custom_prediction_routines/SDK_Pytorch_Custom_Predict.ipynb